In [1]:
#Import libraries
import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer
import numpy as np
import nltk
sia = SentimentIntensityAnalyzer()

In [2]:
#Open the dataset and check the first lines of code
df_business = pd.read_parquet("df_business.parquet")

In [3]:
ny_count = (df_business['state'] == 'NY').sum()
print(ny_count)

0


In [4]:
unique_states = df_business['state'].unique()
print(unique_states)

['CA' 'FL' 'PA' 'TX']


In [5]:
df_users = pd.read_csv('df_user.csv')
df_users.head()

,Unnamed: 0,user_id,name,review_count,useful,id_user_yelp
0,0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,7217,0
1,1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,43091,1
2,2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2086,2
3,3,SZDeASXq7o05mMNLshsdIA,Gwen,224,512,3
4,4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,29,4


In [6]:
df_reviews = pd.read_parquet('df_reviews.parquet')
df_reviews.head()

,business_id,review_id,user_id,stars,text,date,id_review_yelp
0,MTSW4McQd7CbVtyjqoe9mw,BXQcBN0iAi1lAUxibGLFzA,6_SpY41LIHZuIaiDs5FMKA,4.0,This is nice little Chinese bakery in the hear...,2014-05-26 01:09:53,0
1,MTSW4McQd7CbVtyjqoe9mw,uduvUCvi9w3T2bSGivCfXg,tCXElwhzekJEH6QJe3xs7Q,4.0,This is the bakery I usually go to in Chinatow...,2013-10-05 15:19:06,1
2,MTSW4McQd7CbVtyjqoe9mw,a0vwPOqDXXZuJkbBW2356g,WqfKtI-aGMmvbA9pPUxNQQ,5.0,"A delightful find in Chinatown! Very clean, an...",2013-10-25 01:34:57,2
3,MTSW4McQd7CbVtyjqoe9mw,MKNp_CdR2k2202-c8GN5Dw,3-1va0IQfK-9tUMzfHWfTA,5.0,I ordered a graduation cake for my niece and i...,2018-05-20 17:58:57,3
4,MTSW4McQd7CbVtyjqoe9mw,D1GisLDPe84Rrk_R4X2brQ,EouCKoDfzaVG0klEgdDvCQ,4.0,HK-STYLE MILK TEA: FOUR STARS\n\nNot quite su...,2013-10-25 02:31:35,4


In [7]:
rest_cat= pd.read_parquet("yelp_rest_cat_final.parquet")

In [8]:
rest_cat.head

<bound method NDFrame.head of                   business_id Predicted_Category
0      -0EdehHjIQc0DtYU8QcAig              Asian
1      -0FX23yAacC4bbLaGPvyxw           American
2      -0dKgi_Hpcis921nOpM85Q           American
3      -0eUa8TsXFFy0FCxHYmrjg               Cafe
4      -0gRYq5UjMtZbELj0KHxzA               Cafe
...                       ...                ...
14670  zy-BlEF0mkvAwOk3ru1WLA           American
14671  zy0fsu1Wns7g6Om0SxU67g           American
14672  zyMkbavgHASQtqVwaock9A               Cafe
14673  zzIF9qp2UoHN48EeZH_IDg           American
14674  zziDpuuJw-Km1J4BaGpBKA             Family

[14615 rows x 2 columns]>

In [9]:
# Merge df_reviews and df_business
combined_df = pd.merge(df_reviews, df_business, on='business_id', how='left')

# Merge the result with rest_cat
combined_df = pd.merge(combined_df, rest_cat, on='business_id', how='left')

# Merge the result with df_users
combined_df = pd.merge(combined_df, df_users, on='user_id', how='left')

In [10]:
# Convert the 'date' column to datetime
combined_df['date'] = pd.to_datetime(combined_df['date'])

# Filter to keep only rows with date from 2016 onwards
combined_df = combined_df[combined_df['date'].dt.year >= 2016]
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1983810 entries, 3 to 3024657
Data columns (total 23 columns):
 #   Column              Dtype         
---  ------              -----         
 0   business_id         object        
 1   review_id           object        
 2   user_id             object        
 3   stars_x             float64       
 4   text                object        
 5   date                datetime64[ns]
 6   id_review_yelp      int64         
 7   name_x              object        
 8   address             object        
 9   city                object        
 10  state               object        
 11  postal_code         object        
 12  latitude            float64       
 13  longitude           float64       
 14  stars_y             float64       
 15  review_count_x      float64       
 16  categories          object        
 17  Predicted_Category  object        
 18  Unnamed: 0          float64       
 19  name_y              object        
 20  review_

In [11]:
combined_df.head(20)

,business_id,review_id,user_id,stars_x,text,date,id_review_yelp,name_x,address,city,...,longitude,stars_y,review_count_x,categories,Predicted_Category,Unnamed: 0,name_y,review_count_y,useful,id_user_yelp
3,MTSW4McQd7CbVtyjqoe9mw,MKNp_CdR2k2202-c8GN5Dw,3-1va0IQfK-9tUMzfHWfTA,5.0,I ordered a graduation cake for my niece and i...,2018-05-20 17:58:57,3,St Honore Pastries,935 Race St,Philadelphia,...,-75.155564,4.0,80.0,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",Cafe,107074.0,Katherine,28.0,40.0,107074.0
5,MTSW4McQd7CbVtyjqoe9mw,_hJu0u6nB-8LIeQJY4Vg4w,KQSRUu4Aapl0hG6eu2v8iw,4.0,This is my favorite bakery in Chinatown! I usu...,2018-03-17 23:47:25,5,St Honore Pastries,935 Race St,Philadelphia,...,-75.155564,4.0,80.0,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",Cafe,48185.0,Lizzie,759.0,1631.0,48185.0
7,MTSW4McQd7CbVtyjqoe9mw,_xRGsS6QGpcD9LQJNtavuw,qUfRCH5NUyRDsJfM6jA5PQ,4.0,Impression: The good thing is that there are l...,2017-04-20 12:37:09,7,St Honore Pastries,935 Race St,Philadelphia,...,-75.155564,4.0,80.0,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",Cafe,73743.0,Sandy,625.0,2869.0,73743.0
16,MTSW4McQd7CbVtyjqoe9mw,MnYtdGlo861h5auUxV2cwg,0q2W3-ieBUJWD5TTLKi3Ug,4.0,"I would give five stars, but I probably didn't...",2016-04-12 17:35:26,16,St Honore Pastries,935 Race St,Philadelphia,...,-75.155564,4.0,80.0,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",Cafe,80102.0,Kate,36.0,45.0,80102.0
17,MTSW4McQd7CbVtyjqoe9mw,3LNKer0gTFgknV-2CyPjAg,z-yvbUGwFn8PAijEHdU_RA,5.0,While I've never tried their bao (and usually ...,2018-03-25 12:35:42,17,St Honore Pastries,935 Race St,Philadelphia,...,-75.155564,4.0,80.0,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",Cafe,121922.0,D,92.0,36.0,121922.0
20,MTSW4McQd7CbVtyjqoe9mw,UrLiacjSwp53jn8tFtcEsA,YqqSMPzBrZIng-Y0YJTvfw,5.0,The steamed pork buns here are out of this wor...,2017-06-21 15:39:14,20,St Honore Pastries,935 Race St,Philadelphia,...,-75.155564,4.0,80.0,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",Cafe,125652.0,Lawrence,5.0,4.0,125652.0
23,MTSW4McQd7CbVtyjqoe9mw,L3j8agVEAwrTI9A5hd0MiQ,yMlk3HBqiRi0-HIU2jtmIw,5.0,My go to place to get a nice birthday cake. No...,2017-04-23 04:44:24,23,St Honore Pastries,935 Race St,Philadelphia,...,-75.155564,4.0,80.0,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",Cafe,120236.0,Elodie,21.0,9.0,120236.0
24,MTSW4McQd7CbVtyjqoe9mw,ibUkH1a01VZgBD9ba6wocw,h-NFrVwhhCvxdle_3pBwiQ,5.0,Got my wedding cake here and it was the best d...,2018-04-12 20:59:21,24,St Honore Pastries,935 Race St,Philadelphia,...,-75.155564,4.0,80.0,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",Cafe,1100.0,Donna,67.0,152.0,1100.0
27,MTSW4McQd7CbVtyjqoe9mw,KrqIxVaurxWTIeHzNt8P-A,4H0chGckIoOzGv2DcK2KSw,3.0,People are really cold and reserved didnt feel...,2016-07-12 23:38:43,27,St Honore Pastries,935 Race St,Philadelphia,...,-75.155564,4.0,80.0,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",Cafe,128833.0,Lauren,6.0,3.0,128833.0
31,MTSW4McQd7CbVtyjqoe9mw,l_2h0QHc8S_NqxIar7lVMQ,MaueOwM1-iPoOaA5F6a5xA,4.0,So I don't know how the other review was doubl...,2017-08-24 04:23:03,31,St Honore Pastries,935 Race St,Philadelphia,...,-75.155564,4.0,80.0,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",Cafe,42297.0,Melissa,17.0,27.0,42297.0


In [12]:
# Converting 'date' column to datetime type
combined_df['date'] = pd.to_datetime(combined_df['date'])

#Converting the date format "%Y-%m"
combined_df['date'] = combined_df['date'].dt.strftime('%Y-%m')

combined_df['date']

3          2018-05
5          2018-03
7          2017-04
16         2016-04
17         2018-03
            ...   
3024653    2019-07
3024654    2021-06
3024655    2016-12
3024656    2020-02
3024657    2021-06
Name: date, Length: 1983810, dtype: object

In [13]:
# Obtaining data only of "restaurant" category
combined_df = combined_df[combined_df['categories'].str.lower().str.contains('restaurant', na=False)].reset_index(drop=True)
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 565511 entries, 0 to 565510
Data columns (total 23 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   business_id         565511 non-null  object 
 1   review_id           565511 non-null  object 
 2   user_id             565511 non-null  object 
 3   stars_x             565511 non-null  float64
 4   text                565511 non-null  object 
 5   date                565511 non-null  object 
 6   id_review_yelp      565511 non-null  int64  
 7   name_x              565511 non-null  object 
 8   address             565511 non-null  object 
 9   city                565511 non-null  object 
 10  state               565511 non-null  object 
 11  postal_code         565511 non-null  object 
 12  latitude            565511 non-null  float64
 13  longitude           565511 non-null  float64
 14  stars_y             565511 non-null  float64
 15  review_count_x      565511 non-nul

In [14]:
def analyze_sentiment(text):
    if isinstance(text, str):
        compound_score = sia.polarity_scores(text)['compound']
        if compound_score >= 0.5:
            return 5
        elif compound_score >= 0:
            return 4
        elif compound_score >= -0.24:
            return 3
        elif compound_score >= -0.4:
            return 2
        else:
            return 1
    else:
        return 3  # Valor predeterminado si no es una cadena

# Vectorizar la función de análisis de sentimientos
vectorized_analyze_sentiment = np.vectorize(analyze_sentiment)

# Aplicar el análisis de sentimientos a la columna 'text'
combined_df['sentiment_analysis'] = vectorized_analyze_sentiment(combined_df['text'])

# Eliminar la columna 'text'
combined_df.drop(columns=['text'], inplace=True)

In [18]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 565511 entries, 0 to 565510
Data columns (total 23 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   business_id         565511 non-null  object 
 1   review_id           565511 non-null  object 
 2   user_id             565511 non-null  object 
 3   stars_x             565511 non-null  float64
 4   date                565511 non-null  object 
 5   id_review_yelp      565511 non-null  int64  
 6   name_x              565511 non-null  object 
 7   address             565511 non-null  object 
 8   city                565511 non-null  object 
 9   state               565511 non-null  object 
 10  postal_code         565511 non-null  object 
 11  latitude            565511 non-null  float64
 12  longitude           565511 non-null  float64
 13  stars_y             565511 non-null  float64
 14  review_count_x      565511 non-null  float64
 15  categories          565511 non-nul

In [ ]:
# Delete the columns 'Unnamed: 0'
combined_df.drop(['Unnamed: 0', "review_count_y", "stars_y","id_user_yelp", "id_review_yelp", "categories", "useful", "user_name" ], axis=1, inplace=True)

In [22]:
#Rename columns
combined_df.rename(columns={'name_x': 'business_name', 'name_y': 'user_name'}, inplace=True)

combined_df.rename(columns={'review_count_x': 'review_count_business', 'review_count_y': 'review_count_user'}, inplace=True)

combined_df.rename(columns={'stars_x': 'stars_business', 'stars_y': 'stars_user'}, inplace=True)

In [21]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 565511 entries, 0 to 565510
Data columns (total 17 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   business_id            565511 non-null  object 
 1   review_id              565511 non-null  object 
 2   user_id                565511 non-null  object 
 3   stars_business         565511 non-null  float64
 4   date                   565511 non-null  object 
 5   business_name          565511 non-null  object 
 6   address                565511 non-null  object 
 7   city                   565511 non-null  object 
 8   state                  565511 non-null  object 
 9   postal_code            565511 non-null  object 
 10  latitude               565511 non-null  float64
 11  longitude              565511 non-null  float64
 12  review_count_business  565511 non-null  float64
 13  Predicted_Category     565323 non-null  object 
 14  user_name              565503 non-nu

In [25]:
combined_df.drop([ "useful", "user_name" ], axis=1, inplace=True)

In [26]:
# Save the new data
ruta_parquet = 'df_yelp.parquet'

# Guardar el DataFrame user_items en el archivo Parquet especificado
combined_df.to_parquet(ruta_parquet, index=False)

print(f'The file was succesfully saved in: {ruta_parquet}')

The file was succesfully saved in: df_yelp.parquet


In [27]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 565511 entries, 0 to 565510
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   business_id            565511 non-null  object 
 1   review_id              565511 non-null  object 
 2   user_id                565511 non-null  object 
 3   stars_business         565511 non-null  float64
 4   date                   565511 non-null  object 
 5   business_name          565511 non-null  object 
 6   address                565511 non-null  object 
 7   city                   565511 non-null  object 
 8   state                  565511 non-null  object 
 9   postal_code            565511 non-null  object 
 10  latitude               565511 non-null  float64
 11  longitude              565511 non-null  float64
 12  review_count_business  565511 non-null  float64
 13  Predicted_Category     565323 non-null  object 
 14  sentiment_analysis     565511 non-nu